In [13]:
#sender public and private key
!openssl genrsa -out private.pem 2048
!openssl rsa -in private.pem -outform DER -out send_priv.der
!openssl rsa -in private.pem -outform DER -pubout -out send_pub.der

Generating RSA private key, 2048 bit long modulus (2 primes)
........

................................

...

..........+++++
....

...................

...

..................

....

..+++++
e is 65537 (0x010001)


writing RSA key


writing RSA key


In [14]:
#reciver public and private key
!openssl genrsa -out private.pem 2048
!openssl rsa -in private.pem -outform DER -out rec_priv.der
!openssl rsa -in private.pem -outform DER -pubout -out rec_pub.der

Generating RSA private key, 2048 bit long modulus (2 primes)
....+++++
..........

....................

....................................

.

..........................

.................+++++
e is 65537 (0x010001)


writing RSA key


writing RSA key


In [15]:
#AES key generation
!openssl rand 16 > aes.key

In [16]:
from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA
from Crypto import Random


def sign_message(message,privkeyfile_der):
    key = RSA.import_key(open(privkeyfile_der,'rb').read())
    h = SHA256.new(message.encode())
    signature = pss.new(key).sign(h)
    return signature


def verify_sig(message,signature,pubkeyfile_der):
    key = RSA.import_key(open(pubkeyfile_der,'rb').read())
    h = SHA256.new(message)
    verifier = pss.new(key)
    try:
        verifier.verify(h, signature)
        print( "The signature is authentic.")
    except (ValueError, TypeError):
        print( "The signature is not authentic.")


def RSAencrypt(message,pubkeyfile_der):
    key = RSA.importKey(open(pubkeyfile_der,'rb').read())
    cipher = PKCS1_OAEP.new(key)  ### Padding Scheme
    ciphertext = cipher.encrypt(message)
    return ciphertext

def RSAdecrypt(ciphertext,privatekey_der):
    key = RSA.importKey(open(privatekey_der,'rb').read())
    cipher = PKCS1_OAEP.new(key)
    message = cipher.decrypt(ciphertext)
    return message

In [17]:
!echo -n "dog is cat and cat is dog" > msg

In [18]:
#Using sender's private key and the plaintext file we create the signature 
#assign it to msg.sig(to be transmitted to the reciever) file

msg = open("msg").read()

signature = sign_message(msg,"send_priv.der")
try:
    fp = open("msg.sig","wb")
    fp.write(signature)
    fp.close()
except:
    print("IO error")
    
signature

b'O\xb7\xa6:\xf0!5\xf6\xfb\x13\xde+_\xbc@\xe0\xe1\xf62<\x8e\xd0\xb0=K\xcau\n%6\x02\xac\x15\xfa\x05\xd9\x93_Ml\xbf\xed\x96\xb3qBYfn\x80\xe3}2\x06\xc2\xd9"\xf8\r\x99\x83\xb1\xb0\x08\x83\xa8_B\x91a\xe3\xcdhE\xf8\x04p\x97\x9fB\x1dmk\xd0\x10Lk\xd6\xceUx\x9d/a\x19&\xcaA\x88\xa6\xdb\x84\x0f\x98\x039\xa0\x89\xfa\xa6\xab\xdcL%s\t\x86+\xd7|\x9b\xcc\x9c\xa7\x05\xe6C~\x941\xdft\x06%\xbeGi\xb1\xc6\x12\xb4\xa1\xe0\x0e\xb5\xbaS`\xbd\xcaN\xbc\x83\xe6\xaf\xfd\xb0w+\xe3\x8e\x85:\xd4\xdd\x8b\x1d_A\xa1\xde\x90&\xc2\xa6\xa0Q3@\x95\x95^{\xe3\x87\xa5\x8e\xe7m\xde\';\xdf\xb3V8[\xfa>\x9fCM\xc1l#nt\xd8\tbu\xb9x\x18\x88\x88\xb9X\xd2,\x03\xa6\x81cv\xaf\x84H\xff\x9f\xb6\xd2\xbe\xadt\xfd6\x80\xeb\x8aQ2s.l\r\x90\xfc\xa8;\xfa\xe4\xcd\x8ey\xbe'

In [21]:
#using AES-128bit in CTR mode we encrypt the plaintext file 
#assign it to msg.crypt file

!openssl aes-128-ctr -pbkdf2 -iv 101112131415161718191A1B1C1D1E1F -e -pass file:aes.key -in msg -out msg.crypt


In [22]:
#Using recievers public key the sender can encrypt the aes key so to securely send it to the reciever 

msg = open("aes.key",'rb').read()

ct = RSAencrypt(msg,"rec_pub.der")
try:
    fp = open("symkey.crypt","wb")
    fp.write(ct)
    fp.close()
except:
    print("IO error")
ct

b'.\xec\xc6\x12\xb5*g\xa4\xb7\x83\xa3\x18n\xf4x\x93\xfa\xe6\x93J\xc6\xc9\x05\x92\xa3\x98\x94\xc8\xf3\xec\x85\x1c\x94\xcc\xfa\xba\x9bds\x89C\xfe\xb8(;\xdd\xc2&\xf2h\xd0\xda\xc6\xbdp\x1f\xbd\x0b\x8f8Z\xf8\xa3\xecC\x02\xd6!\xe3)\x07\x99\x8d\x90f\xd7\x82\xacj\xf6\xac1\xfe\xd04\xc23Y\x9c\xec\x1d\x95\xbcl\x1f\xb6oR\x14\xd9\x8a\x0b\xf8\xcdA\x19\xae\x81\xf2\xcc1Q\n\\\xcf:\xadl\xb7q\xb6\xee\xfa\xa3\xc8\x17\xdb0\xd4\xd6`)8\x17\xfe]\xad:g\xbbY\xe3X\xe6\x9d\xe72\xaf\x1f6i~k\xe3\xcc1A\x93\x12/\xd4cj\xf7B\x1eS\xf2\x15\xae}\x9b\xd5\x92\x1f\xf2\xfc\xcc\xc0\xe0\x187\xabW\x8e\x08<\xa7\xb4i\x04S)6U\x0e\xa2G\xcb@\xfc\xe3\xd5yV\x82\x9f\xad/H\x02\xf86\x11ip\xed\xb6*\x04\xb5\x08\xad\xd4Y\xa2\x08\xb5E\x91d\xb4]\t!\xf36\x04t{L\xf8e.\xc5\xd1\xa3\x7f\x93\x85^\r\x95\x99\xaa\x04'

In [23]:
#decrypt symkey.crypt(RSA encrypted AES key) to produce symkey(AES key) file

ct1 = open("symkey.crypt", "rb").read()
symkey = RSAdecrypt(ct1,"rec_priv.der")

try:
    file = open("rec_aes.key", "wb")
    file.write(symkey)
    file.close()
except:
    print("IO error")
symkey

b']\xc9\xd4\x1b\x9cW\x8d\x98\xba\xc2\xa7B\xd3\xd5\xbe%'

In [24]:
#Using AES key(symkey) to decrypt msg.crypt(sent via sender) to msg(plaintext)

!openssl aes-128-ctr -pbkdf2 -iv 101112131415161718191A1B1C1D1E1F -d -pass file:rec_aes.key -in msg.crypt -out msg

In [25]:
#Receiver verifies msg file signature using msg.sig(signature file)
#to make sure the content has not been tampered with

recsigfile= open("msg.sig", "rb").read()
rec_msg = open("msg", "rb").read()

verify_sig(rec_msg, recsigfile,"send_pub.der")

The signature is authentic.
